# Black-Scholes - Monte Carlo Simulation - No dividends

> Author: Gustavo Monteiro de Athayde  
> Date: 15/03/2025



In [17]:
# dependencies
from IPython.display import display, clear_output

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Brigde part

In [ ]:
# BS parameters
S0 = 100        # spot price
K = 100         # strike price
T = 1           # time to maturity
rf = 0.00       # risk-free rate
vol = 0.4       # volatility

In [ ]:
# Monte Carlo simulation parameters
n = 1000000        # number of paths  - 5 min
n = 100000

# underlying parameters
mimc = (rf - vol**2/2)*(T/252)          # drift
vol_mc = vol/np.sqrt(252)
mi = rf - vol**2/2                      # drift (mi)

## Naive approach

In [16]:
# let's simulate the paths - most dummy way
np.random.seed(0)
dt = T/n
t = np.linspace(0, T, n)
S = np.zeros((n, 252))

for i in range(n):
    S[i, 0] = S0
    for j in range(1, 252):
        S[i, j] = S[i, j-1]*np.exp(mimc + vol_mc*np.random.normal(0, 1))

In [13]:
S.T

In [18]:
# plot the paths	
plt.figure(figsize=(10, 6))
plt.plot(S.T)
plt.title('Simulated paths')

## Monte Carlo Simulation using pandas

In [1]:
# let simulate the stock price trajectory using pandas
np.random.seed(0)
dt = T/n    
t = np.linspace(0, T, n)
S = pd.DataFrame(S)
S.columns = t
S.index = range(1, n+1)
S
